这节课我们来讲一些具体的目标检测算法，如R-CNN、SSD、YOLO等

首先是R-CNN
* R-CNN首先通过一种启发式的搜索方法（挺复杂的一个算法），来选择一些锚框
    * 所谓的启发式算法，就是通过一些经验原则来选择局部最优解（通常不会是全局最优解），比如贪心算法
* 选好锚框后，将一个锚框视为一个单独的图片，然后这个图片扔进CNN去提取特征
* 对提取出来的特征，用一个SVM去训练类别分类器，用一个linear去训练锚框与真实边缘框的距离
* 这里会碰到一个实际上的问题，也就是我们每次选取的锚框很有可能不一样大，那我这个不同锚框之间咋去合成一个batch呢？
* 这里引入了一种方法 ROI pooling，也就是在paper里面很常见的ROI
* 也就是给定的锚框，我将其切为n*m大小的块，然后取每个块里面的最大值，直观的能感受到这东西起到了一个标准化的作用，不管你锚框多大，那我总输出nm个值

R-CNN的一个问题就是，对一张图片我们可能能产生上千个锚框，那原始的R-CNN的算法我们这样要做上千次的CNN特征提取，这样太难搞了
* 接下来我们提出了Fast-RCNN
* 首先我们对一张图片，将其视为一个样本，应用CNN直接抽特征，这里和CNN直接做图像分类其实是很像的
* 对于原始图片，我们做和R-CNN里面一样的锚框搜索，得到若干个锚框
* 对于CNN抽出来的结果，其实就是一个小一点的矩阵（也就是feature map），我们将上面得到的锚框按照CNN抽出来的feature map比例跟他放进去，映射到feature map上面
* 映射完之后，我们用ROI pooling，对feature map上面的锚框进行pooling
* 再进全连接层，做类别预测，偏差预测

在Fast-RCNN之后，新的套娃Faster-RCNN出现了，有以下改进：
* 我们用神经网络来替代前面提到的最开始选择性启发式搜索锚框的方法，叫区域提议网络region proposal network
* 用上面提到的CNN提取特征得到的feature map，
* 对CNN拿到的feature map，再经过一次conv layer，等于再提取一次特征，然后用某一方法（可以是启发式搜索，或者是之前我们提到的基于中心点算锚框的方法）得到很多锚框
* 这些锚框其实可以视作一个比较粗糙的物体检测结果，来做预测器
    * 这里的预测器和普通的锚框预测存在一点差别
    * 我们这里的类别预测是一个二分类问题，也就是是否包括了真实物体（包括了就是高质量锚框）
    * 然后还要预测偏差值
* 用这些高质量锚框，也就是效果不那么好的物体检测结果，去给后面的网络用
* 然后针对前面得到的这些锚框，我们去做之前提到的ROI pooling，然后放全连接层做预测
* Faster其实是现在比较常用的一个东西

基于前面的，Mask-RCNN出现了：
* Mask-RCNN需要一个具有每个像素标号信息的输入，这个涉及到后面的语义分割，到时候再具体的讲

统计一下会发现，Faster-RCNN不考虑速度的情况下，具有最好的精度，一般你不考虑速度的话，我们用Faster、Mask

接下来是SSD，虽然这玩意现在已经不太做了
* 前面R-CNN我们会发现一个问题，就是一个数据进来其实我们要做两遍，比如一个主要的网络和一个Faster里面的RPN，SSD就说我不做两遍，我只做一遍
* SSD对于每个像素来说，按照它为中心生成若干个锚框，和我们介绍锚框生成里面的那个算法是一样的，可以往回看一下
* SSD的组做法就是，一张图片进来，我先通过一个卷积来抽取特征，抽取出来的特征我去做上面的锚框生成，然后根据这些锚框去做预测
* 上面我们这个步骤，重复n次，也就是有n次特征抽取与生成锚框
* 在CNN里面我们知道，我们一般来说越往上卷积层越小，提取的越是关键信息
* SSD就认为是，我越往上我越抽取到的就是小物体的检测，而下面是大物体的检测
* SSD相对来说速度很快，但是相对来说精度就一般了，因为这玩意提出来之后就没怎么更新了

大名鼎鼎的YOLO其实是受到了SSD的启发
* YOLO强调的是快
* 上面我们提到过，SSD这种算法会产生大量大量大量的锚框，YOLO觉得这样太浪费了，所以他将图片进行简单的快装切割，直接一张图片生成若干个不想交的锚框
* 对于这些锚框来说，我每个锚框可以预测n个真实的边缘框，这是因为上面这样的切割方式太暴力，可能你一个锚框会刚好跨在两个真实的物体上
* YOLO比SSD还要快

目标检测算法主要分为两个类型
1. two-stage方法，如R-CNN系算法（region-based CNN），其主要思路是先通过启发式方法（selective search）或者CNN网络（RPN)产生一系列稀疏的候选框，然后对这些候选框进行分类与回归，two-stage方法的优势是准确度高
2. one-stage方法，如Yolo和SSD，其主要思路是均匀地在图片的不同位置进行密集抽样，抽样时可以采用不同尺度和长宽比，然后利用CNN提取特征后直接进行分类与回归，整个过程只需要一步，所以其优势是速度快，但是均匀的密集采样的一个重要缺点是训练比较困难，这主要是因为正样本与负样本（背景）极其不均衡，导致模型准确度稍低